# 전이학습 활용: 영상 분류

- source:  https://www.tensorflow.org/guide/keras/transfer_learning?hl=ko
- modified by BY Kim

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 
import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from sklearn.metrics import confusion_matrix, accuracy_score


PATH = '/content/drive/MyDrive/ICTIS_2023/data/'  

tf.__version__

'2.12.0'

## 개 & 고양이 데이터 셋 가져오기

In [2]:
import tensorflow_datasets as tfds

tfds.disable_progress_bar()

train_ds, validation_ds, test_ds = tfds.load(
    "cats_vs_dogs",
    # Reserve 10% for validation and 10% for test
    split=["train[:40%]", "train[40%:50%]", "train[50%:60%]"],
    as_supervised=True,  # Include labels
)

print("Number of training samples: %d" % tf.data.experimental.cardinality(train_ds))
print(
    "Number of validation samples: %d" % tf.data.experimental.cardinality(validation_ds)
)
print("Number of test samples: %d" % tf.data.experimental.cardinality(test_ds))

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.
Number of training samples: 9305
Number of validation samples: 2326
Number of test samples: 2326


## 데이터 시각화
- Dog: 1, Cat: 0

In [ ]:
plt.figure(figsize=(10, 10))
for i, (image, label) in enumerate(train_ds.take(25)):
    ax = plt.subplot(5, 5, i + 1)
    plt.imshow(image.numpy().astype("uint8"))
    plt.title(f"{int(label):d} ({image.shape[0]:d}, {image.shape[1]:d})")
    plt.axis("off")

## 데이터 증강
- https://www.tensorflow.org/api_docs/python/tf/keras/layers/RandomZoom

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(height_factor = 0.1, width_factor=0.1), 
        layers.RandomBrightness(factor=0.1)
    ]
)

## 데이터 생성  map( )

In [ ]:
image_size = (224, 224)
batch_size = 4

train_ds = train_ds.map(lambda x, y: (tf.image. (x, image_size), y))
validation_ds = validation_ds.map(lambda x, y: (tf.image. (x, image_size), y))
test_ds = test_ds.map(lambda x, y: (tf.image. (x, image_size), y))

## 성능 개선을 위한 배치화

In [ ]:
train_ds = train_ds.cache().batch(batch_size).prefetch(buffer_size=batch_size)
validation_ds = validation_ds.cache().batch(batch_size).prefetch(buffer_size=batch_size)
test_ds = test_ds.cache().batch(batch_size).prefetch(buffer_size=batch_size)

## 증강된 영상 보기

- https://www.tensorflow.org/tutorials/load_data/images

In [ ]:
for images, labels in train_ds.take(3) :  # batch = 4

    augmented_images = (images)
    plt.figure(figsize=(8, 3))
    for i in range(4):
        ax = plt.subplot(1, 4, i+1)
        plt.imshow(augmented_images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")
    plt.show()
    
print(images.shape, augmented_images.shape)  

In [ ]:
# 12 types of augmentation for 1 image

for images, labels in train_ds.take(1):
    plt.figure(figsize=(8, 6))
    img_no = 0
    img = images[img_no]
    for i in range(12):
        augmented_images = data_augmentation(
            tf.expand_dims(img, 0), training=True)
        ax = plt.subplot(3, 4, i+1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.title(int(labels[img_no]))
        plt.axis("off")
    plt.show()
print(images.shape, augmented_images.shape)

## 모델 생성

In [ ]:
base_model=tf.keras.applications. (include_top= , 
                                                weights= ,
                                                input_shape= ) 


# Freeze the base_model
base_model.trainable =  
output = base_model 

# Create new model on top

x = tf.keras.layers.GlobalAveragePooling2D()(output)
x = tf.keras.layers.Dropout(rate=.45)(x)   
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(rate=.45)(x)        
output = tf.keras.layers.Dense( , activation= )(x)

model = tf.keras.Model(inputs=base_model.input, outputs=output)

In [ ]:
base_model.summary()

In [ ]:
model.summary()

## 학습 데이터 셋에 대한 데이터 증강 적용

In [ ]:
# Apply `data_augmentation` to the training images.
train_ds = train_ds.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf.data.AUTOTUNE
)
# Prefetching samples in GPU memory helps maximize GPU utilization.
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

## 모델 학습

In [ ]:
model.compile(optimizer= keras.optimizers.Adam(), 
              loss=keras.losses.BinaryCrossentropy(),
              metrics=['accuracy']) 
  
# 모델 저장 조건 설정
modelfile = PATH+ 'TL_ResNet50_CatDog_best.h5'  
checkpointer = ModelCheckpoint(filepath=modelfile, 
                               monitor='val_loss', verbose=1, 
                               save_best_only=True)

# 학습의 자동 중단 설정
early_stopping_callback = EarlyStopping(monitor='val_loss',
                                        patience=10)


fit_history=model.fit(train_ds, validation_data = validation_ds,
                      callbacks=[early_stopping_callback, checkpointer],
                        epochs=50, verbose=1)

We get to ~96% validation accuracy after training for 50 epochs on the full dataset.

In [ ]:
#학습 이력 확인하기
def display_training_curves(training, validation, title, subplot):    
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.grid(visible=True)
    ax.set_facecolor('#D0D0D0')
    if subplot%10==1:   # loss
        tmin, tmax = min(training), max(training)
        vmin, vmax = min(validation), max(validation) 
        min_val =  tmin if tmin<vmin else vmin
        max_val = tmax  if tmax>vmax else vmax
        ax.set_ylim(min_val - 0.1, max_val + 0.1)
    else:   # accuracy
        ax.set_ylim(0.4, 1.0)        
    ax.plot(training, marker='o')
    ax.plot(validation, marker='o')
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])
    
def display_training_history(fit_history):
    display_training_curves(
        fit_history.history['loss'], 
        fit_history.history['val_loss'], 
        'loss', 211)
    display_training_curves(
        fit_history.history['accuracy'], 
        fit_history.history['val_accuracy'], 
        'accuracy', 212)

In [ ]:
display_training_history(fit_history)

In [ ]:
type(test_ds)

In [ ]:
test_ds

In [ ]:
optimal_model = keras.models.load_model(PATH+'TL_ResNet50_CatDog_best.h5')

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Apply `data_augmentation` to the testing images.
test_ds = test_ds.map(
    lambda  :   ,
    num_parallel_calls=tf.data.AUTOTUNE
)
# Prefetching samples in GPU memory helps maximize GPU utilization.

test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
for images, labels in  test_ds :

    y_true = labels.numpy().astype(int).flatten()
    predictions = optimal_model.predict(images)   
    

    plt.figure(figsize=(8, 3))
    for i in range(4):
        ax = plt.subplot(1, 4, i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f"{int(y_pred[i]):d} ({y_true[i]:d}) ")
        plt.axis("off")
    plt.show()

    print(accuracy_score(y_true, y_pred))
    print(confusion_matrix(y_true, y_pred))

## Fine Tuning

In [ ]:
model = keras.models.load_model(PATH+'TL_ResNet50_CatDog_best.h5')
model.summary()

In [ ]:

model.summary()

In [ ]:

model.compile(optimizer= keras.optimizers.Adam(  ),  # Very low learning rate
              loss=keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

fit_history = model.fit(train_ds, epochs= 3, 
          validation_data = validation_ds, verbose=1)

In [ ]:
model.save(PATH+'TL_ResNet50_CatDog_FineTuned.h5')